In [1]:
import cv2
import mediapipe as mp
import numpy as np
import math

# hand detection
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5)

# webcam
cap = cv2.VideoCapture(0)

def calculate_gesture(landmarks):
    thumb_tip = landmarks[mp_hands.HandLandmark.THUMB_TIP.value]
    index_tip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP.value]
    middle_tip = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_TIP.value]

    # pinching gesture
    distance = math.hypot(thumb_tip.x - index_tip.x, thumb_tip.y - index_tip.y)
    # global gesture_state
    # if gesture_state["pinch"] == "Pinched":
    #     if distance > 0.08: # Adjust this threshold for release sensitivity
    #         if gesture_state["pinch"] == "Pause":
    #             gesture_state["pinch"] = "Play"
    #             print("Play music")
    #         else:
    #             gesture_state["pinch"] = "Pause"
    #         print({gesture_state["pinch"]}, " music")
    #         return gesture_state["pinch"]
    # elif distance < 0.05:
    #     gesture_state["pinch"] = "Pinched"


    # if gesture_state["pinch"] == "Pinched":
    #     prev_pinch_y = gesture_state.get("prev_pinch_y", 0.5)
    #     distance = math.hypot(thumb_tip.x - index_tip.x, thumb_tip.y - index_tip.y)
    #     if distance > 0.08 and thumb_tip.y < prev_pinch_y - 0.03:
    #         gesture_state["prev_pinch_y"] = thumb_tip.y
    #         print("Volume Up condition met")
    #         return "Volume Up"
    #     gesture_state["prev_pinch_y"] = thumb_tip.y

    # if gesture_state["pinch"] == "Pinched":
    #     prev_pinch_y = gesture_state.get("prev_pinch_y", 0.5)
    #     distance = math.hypot(thumb_tip.x - index_tip.x, thumb_tip.y - index_tip.y)
    #     if distance > 0.08 and thumb_tip.y > prev_pinch_y + 0.03:  # Downward movement detected
    #         gesture_state["prev_pinch_y"] = thumb_tip.y
    #         print("Volume Down condition met")
    #         return "Volume Down"
    #     gesture_state["prev_pinch_y"] = thumb_tip.y


    # open hand
    if thumb_tip.y < index_tip.y and thumb_tip.y < middle_tip.y:
        return "Play"

    # closed hand
    if thumb_tip.y > index_tip.y and thumb_tip.y > middle_tip.y:
        return "Pause"

    # L (volume control)
    if thumb_tip.x < index_tip.x:   return "Volume Up"
    if thumb_tip.x > index_tip.x:  return "Volume Down"

    # gun shape (not working perfectly)
    if index_tip.y < middle_tip.y:
        if index_tip.x < middle_tip.x:
            return "Previous"
        if index_tip.x > middle_tip.x:
            return "Next"

    return "No Gesture Detected"

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Error reading frame from webcam.")
        continue

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            gesture = calculate_gesture(hand_landmarks.landmark)
            cv2.putText(image, gesture, (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                        1, (0, 255, 0), 2)

    cv2.imshow('Gesture Recognition', image)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()